<a href="https://colab.research.google.com/github/Veldora00/Veldora00/blob/main/yfinancefib_idnetifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch Historical Data
def get_forex_data(pair, start, end):
    data = yf.download(pair, start=start, end=end, interval='1d')
    data = data[['Open', 'High', 'Low', 'Close']]
    return data

# Identify Pivot Highs and Lows
def identify_pivots(data, left=20, right=1):
    pivots_high = []
    pivots_low = []

    for i in range(left, len(data) - right):
        is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
                        all(data['High'][i] > data['High'][i + j] for j in range(1, right + 1))
        is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
                       all(data['Low'][i] < data['Low'][i + j] for j in range(1, right + 1))

        if is_pivot_high:
            pivots_high.append((data.index[i], data['High'][i]))
        if is_pivot_low:
            pivots_low.append((data.index[i], data['Low'][i]))

    return pd.DataFrame(pivots_high, columns=['Date', 'Pivot High']), pd.DataFrame(pivots_low, columns=['Date', 'Pivot Low'])

# Calculate Fibonacci Levels with ±0.2% Tolerance
def calculate_fibonacci_levels(pivot_low, pivot_high):
    levels = {
        '23.6%': pivot_high - 0.236 * (pivot_high - pivot_low),
        '38.2%': pivot_high - 0.382 * (pivot_high - pivot_low),
        '50.0%': pivot_high - 0.500 * (pivot_high - pivot_low),
        '61.8%': pivot_high - 0.618 * (pivot_high - pivot_low),
        '78.6%': pivot_high - 0.786 * (pivot_high - pivot_low),
    }
    return levels

def calculate_fib_tolerance(levels, tolerance_percent=0.002):
    fib_tolerance = {}
    for level_name, level_price in levels.items():
        tolerance_value = level_price * tolerance_percent
        fib_tolerance[level_name] = {
            'Level': level_price,
            'Upper Bound': level_price + tolerance_value,
            'Lower Bound': level_price - tolerance_value
        }
    return fib_tolerance

# Check for pairs near Fibonacci levels
def check_fib_levels(data, pivots_high, pivots_low, pair):
    last_fib_levels = {}

    if pivots_high.empty or pivots_low.empty:
        return last_fib_levels

    for i in range(1, min(len(pivots_high), len(pivots_low))):
        pivot_high_prev = pivots_high.iloc[i-1]['Pivot High']
        pivot_low_prev = pivots_low.iloc[i-1]['Pivot Low']

        fib_levels = calculate_fibonacci_levels(pivot_low_prev, pivot_high_prev)
        fib_tolerance_levels = calculate_fib_tolerance(fib_levels)

        for j in range(1, len(data)):
            close_price = data['Close'].iloc[j]
            for level_name, bounds in fib_tolerance_levels.items():
                if bounds['Lower Bound'] <= close_price <= bounds['Upper Bound']:
                    last_fib_levels[pair] = {
                        'Date': data.index[j],
                        'Pair': pair,
                        'Level': level_name,
                        'Price': close_price,
                        'Fib Level': fib_levels[level_name]
                    }

    return last_fib_levels

# Main function to check all pairs
def analyze_forex_pairs(pairs, start_date, end_date):
    final_results = []
    for pair in pairs:
        print(f"Analyzing {pair}...")
        forex_data = get_forex_data(pair, start_date, end_date)
        pivots_high, pivots_low = identify_pivots(forex_data)
        last_fib_levels = check_fib_levels(forex_data, pivots_high, pivots_low, pair)

        if last_fib_levels:
            final_results.append(last_fib_levels[pair])

    results_df = pd.DataFrame(final_results)

    # Sort the DataFrame by Date in ascending order
    results_df = results_df.sort_values(by='Date', ascending=True)

    if not results_df.empty:
        print(results_df)
    else:
        print("No pairs near Fibonacci levels.")

# Define the pairs to analyze
pairs = [
    'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'USDCHF=X', 'USDCAD=X',
    'EURGBP=X', 'EURCHF=X', 'EURAUD=X', 'EURNZD=X', 'EURCAD=X', 'EURJPY=X',
    'GBPJPY=X', 'GBPCHF=X', 'GBPAUD=X', 'GBPNZD=X', 'GBPCAD=X',
    'AUDJPY=X', 'AUDNZD=X', 'AUDCHF=X', 'AUDCAD=X',
    'NZDJPY=X', 'NZDCHF=X', 'NZDCAD=X',
    'CADJPY=X', 'CADCHF=X',
    'CHFJPY=X'
]

# Run the analysis
start_date = '2023-01-01'
end_date = '2024-08-31'
analyze_forex_pairs(pairs, start_date, end_date)


Analyzing EURUSD=X...


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-40dd30f49477>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

Analyzing GBPUSD=X...


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-40dd30f49477>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

Analyzing AUDUSD=X...


...

Analyzing CADCHF=X...



<ipython-input-1-40dd30f49477>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all(data[

Analyzing CHFJPY=X...


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-40dd30f49477>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_high = all(data['High'][i] > data['High'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  is_pivot_low = all(data['Low'][i] < data['Low'][i - j] for j in range(1, left + 1)) and \
<ipython-input-1-40dd30f49477>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

         Date      Pair  Level       Price   Fib Level
26 2024-05-03  CHFJPY=X  78.6%  168.209000  168.107702
16 2024-07-01  GBPCAD=X  78.6%    1.729310    1.727676
10 2024-07-11  EURCAD=X  38.2%    1.475100    1.474284
25 2024-07-15  CADCHF=X  78.6%    0.657120    0.657075
19 2024-07-24  AUDCHF=X  78.6%    0.589420    0.590481
4  2024-07-24  USDCHF=X  78.6%    0.891520    0.891164
24 2024-07-31  CADJPY=X  61.8%  110.226997  110.086729
13 2024-07-31  GBPCHF=X  78.6%    1.132250    1.134466
20 2024-08-14  AUDCAD=X  23.6%    0.910046    0.909859
0  2024-08-16  EURUSD=X  23.6%    1.097273    1.097205
1  2024-08-19  GBPUSD=X  23.6%    1.294716    1.295695
22 2024-08-20  NZDCHF=X  78.6%    0.527570    0.528303
17 2024-08-20  AUDJPY=X  61.8%   98.794998   98.959374
6  2024-08-21  EURGBP=X  78.6%    0.853680    0.854899
5  2024-08-23  USDCAD=X  78.6%    1.360340    1.359242
3  2024-08-23  NZDUSD=X  38.2%    0.614360    0.615221
2  2024-08-23  AUDUSD=X  23.6%    0.670820    0.671980
7  2024-08